# VCG

Assume $n = 4$ items and $m = 5$ bidders. We solve the VCG mechanism for the most general (and harder) case of combinatorial auctions, and non additive bidders' evalutions. The other cases are easier case of this general case and will follow directly.

In [ ]:
# Bidders' evaluations
v = {
    'a': {
        frozenset({'A'}):2,
        frozenset({'A', 'B'}):5,
        frozenset({'B'}):2,
        frozenset({'D'}):7
    },
    'b':{
        frozenset({'C'}): 10,
        frozenset({'D'}):8
    },
    'c':{
        frozenset({'C', 'D'}):15,
        frozenset({'D'}):10
    },
    'd':{
        frozenset({'B', 'C'}):10,
        frozenset({'B'}):8
    },
    'e':{
        frozenset({'A', 'B'}):11,
        frozenset({'A', 'B', 'C', 'D'}):20
    }
}

In [ ]:
v

{'a': {frozenset({'A'}): 2,
  frozenset({'A', 'B'}): 5,
  frozenset({'B'}): 2,
  frozenset({'D'}): 7},
 'b': {frozenset({'C'}): 10, frozenset({'D'}): 8},
 'c': {frozenset({'C', 'D'}): 15, frozenset({'D'}): 10},
 'd': {frozenset({'B', 'C'}): 10, frozenset({'B'}): 8},
 'e': {frozenset({'A', 'B'}): 11, frozenset({'A', 'B', 'C', 'D'}): 20}}

# The building blocks of the VCG mechanism for combinatorial auctions
Recall that in combinatorial auctions bidders place evaluations for bundle of objects.
In auctions we have to make two choices:
1. Deciding how to allocate the goods
2. Deciding how much every bidder has to pay

In combinatorial auctions we allocate the goods in the way that maximizes the social welfare; that is, among all possible *admissible* allocations $(\Omega)$ we choose the one for which the sum of the player values is maximum. Formally:

$$ \omega^* = \argmax_{\omega \in \Omega} \sum_{i \in N} v_i(\omega) $$

This is an NP-Hard problem, so we have to enumerate all possible admissible allocation.
An allocation is *admissible* if it satisfies two conditions:
- Every bidder gets *at most* one bundle
- The allocated bundles are pairwise disjoint (we cannot allocate the same object to two people)

Next, in order to decide how much each player should pay, we need to compute the externality that each player imposes on all the others.
That is, once we have computed the optimal allocation discussed above $(\omega^*)$, for each player we need to compute the following quantity:

$$ p_i(\omega^*) = \max_{\omega \in \Omega} \sum_{j \neq i \in N} v_j(\omega) - \sum_{j \neq i} v_j(w^*)$$

In short, for each player $i$ we need to compute the best possible admissible allocation if $i$ did not participate to the auction (using the same algorithm we use to solve the prior problem) and subtract the sum of the values of all the players $j \neq i$ in the optimal allocation $\omega^*$.

In [ ]:
import copy as cp

def from_dict_to_list_bids(bids):
    '''
    This function converts the dictionary of bidders' evaluation into a list of tuples
    '''
    l = [(bidder, itemset, value) for bidder in bids for itemset,value in bids[bidder].items()]
    return l

def is_admissible(allocation):
    '''
    This function returns True if the allocation is admissible and False otherwise.

    An allocation is admissible if and only if the following two conditions are satisfied:
        1) every bidder gets at most one bundle
        2) the allocated bundles are disjoint
    '''

    bidders = []
    sets = []
    for bid in allocation:
      bidder = bid[0]
      if bidder in bidders:
        return False
      bidders.append(bidder)
      sets.append(bid[1])

    if len(sets) > 0 and not pairwise_disjoint(sets):
      return False
    return True

def pairwise_disjoint(sets):
    '''
    This function checks whether the sets contained in a list are pairwise disjoint
    '''
    intersection = frozenset()
    already_assigned = sets[0]
    for elem in sets[1:]:
      intersection = already_assigned.intersection(elem)
      already_assigned = already_assigned.union(elem)
      if len(intersection) > 0:
        return False

    return True


def compute_allocation_value(allocation):
    '''
    An allocation is a list of tuples with the following semantic: (bidder, itemset, value).
    This function computes the total value of an allocation.
    '''
    value = 0
    for bid in allocation:
      value += bid[2]
    return value

def compute_winner(bids):
    '''
    This function computes the winner(s) of a combinatorial auction
    Determining the winner(s) of a combinatorial auction, under the VCG mechanism amounts to solving the
    weighted set packing problem, which is NP-Hard. Here, in order to solve this problem, we write an algorithm
    that enumerates all the possible assignments and selects the admissible assignment with the maximum
    social welfare.

    This implementation is not the most efficient possible, since it generates all the possible allocations
    including non admissible allocations. This problem can be mitigated through techniques such as
    constraint propagation, recursive search strategies, and backtracking.

    You're very welcome to try to make an optimized algorithm for this problem ;)
    '''

    # unfold the dict
    bids_list = from_dict_to_list_bids(bids)

    n = len(bids_list)

    winner_value = 0

    for i in range(0,2**n):
      candidate_allocation = []
      bin_i = str(bin(i)).split('b')[1][::-1]   # little endian representation

      for j in range(len(bin_i)):
        if bin_i[j] == '1':
          candidate_allocation.append(bids_list[len(bids_list)-1-j])
      if is_admissible(candidate_allocation):
        candidate_allocation_value = compute_allocation_value(candidate_allocation)
        # check if this allocation has more value than the current winner
        if candidate_allocation_value >= winner_value:
          winner = candidate_allocation
          winner_value = candidate_allocation_value

    return winner, winner_value

def VCG_auction(bids):
    '''
    This function implements the VCG mechanism. It takes in input the bids for the auction and provides in output:
        1) the optimal allocation of the items
        2) the price each winner has to pay

    The format of the output is a list of tuples with the following semantic: (bidder, itemset, price).
    Example:
                ('a', {'I1', 'I2'}, 30) --> Bidder 'a' receives the bundle {'I1', 'I2'} and pays 30

    In VCG mechanism, the optimal allocation is the allocation which maximises the social welfare.That is:

                $$ \omega^* = \\argmax_{\omega \in \Omega} \sum_{i \in N} v_i(\omega) $$

    The amount each player pays is the externality they impose on all the other players. That is:

                $$ p_i(\omega^*) = \max_{\omega \in \Omega} \sum_{j \\neq i \in N} v_j(\omega) - \sum_{j \\neq i} v_j(w^*)$$

    '''
    vcg = []
    omega_star, winner_value = compute_winner(bids)

    for player in bids:
      bids_without_player = cp.deepcopy(bids)
      del bids_without_player[player]

      vcg_itemset = []

      # first part of the expression
      _, v_omega_minus_player = compute_winner(bids_without_player)

      # calculate second part of the expression
      summation_omega_star_minus_player = 0
      for bidder, itemset, value in omega_star:
        if bidder != player:
          summation_omega_star_minus_player += value
        else:
          vcg_itemset.append(itemset)

      vcg_value = v_omega_minus_player - summation_omega_star_minus_player

      vcg.append((player, vcg_itemset, vcg_value))

    return vcg

In [ ]:
compute_winner(v)

([('e', frozenset({'A', 'B'}), 11),
  ('c', frozenset({'D'}), 10),
  ('b', frozenset({'C'}), 10)],
 31)

In [ ]:
VCG_auction(v)

[('a', [], 0),
 ('b', [frozenset({'C'})], 5),
 ('c', [frozenset({'D'})], 7),
 ('d', [], 0),
 ('e', [frozenset({'A', 'B'})], 10)]